# 🎌 Anime 4K Upscale v2.2 - STABLE

**Bản sửa lỗi cắt file. Chạy ổn định hơn.**

1. Sửa `INPUT_DIR`
2. **Runtime → Restart runtime**
3. **Runtime → Run all** (`Ctrl+F9`)

In [ ]:
#@title ⚙️ CẤU HÌNH
INPUT_DIR = "/content/drive/MyDrive/Movie/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movie/Output" #@param {type:"string"}
MODE = "anime_4k" #@param ["anime_4k", "anime_fast", "ultra_fast"]

SEGMENT_TIME = 900 #@param {type:"integer"}
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}

In [ ]:
#@title 🚀 CHẠY (Stable)
import os, glob, subprocess, json, time, pickle, site, math, shutil
from datetime import timedelta

os.chdir('/content')

# Mount
print("📂 Mounting Drive...")
from google.colab import drive
drive.mount('/content/drive')

print("\n🖥️ GPU:")
!nvidia-smi --query-gpu=name,memory.free --format=csv

# ====== PATCH BASICSR ======
print("\n🔧 Patching basicsr...")
patched = False
for sp in site.getsitepackages() + ['/usr/local/lib/python3.12/dist-packages']:
    pf = os.path.join(sp, 'basicsr/data/degradations.py')
    if os.path.exists(pf):
        with open(pf, 'r') as f: c = f.read()
        if 'functional_tensor' in c:
            c = c.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale', 'from torchvision.transforms.functional import rgb_to_grayscale')
            with open(pf, 'w') as f: f.write(c)
            print(f"   ✓ basicsr patched: {pf}")
            patched = True

if not patched:
    print("   ⚠️ Installing basicsr...")
    !pip install basicsr -q
    for sp in site.getsitepackages():
        pf = os.path.join(sp, 'basicsr/data/degradations.py')
        if os.path.exists(pf):
            with open(pf, 'r') as f: c = f.read()
            c = c.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale', 'from torchvision.transforms.functional import rgb_to_grayscale')
            with open(pf, 'w') as f: f.write(c)
            print(f"   ✓ Patch retry success")

# ====== INSTALL ======
print("\n📦 Setup Real-ESRGAN...")
ESRGAN = '/content/Real-ESRGAN'
MODEL = f'{ESRGAN}/weights/realesr-animevideov3.pth'

if not os.path.exists(MODEL):
    !rm -rf {ESRGAN}
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    os.chdir(ESRGAN)
    !pip install facexlib gfpgan -q
    !pip install -r requirements.txt -q
    !python setup.py develop 2>&1 | tail -1
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q

# ====== PATCH INFERENCE (Fix nb_frames) ======
inf_file = f'{ESRGAN}/inference_realesrgan_video.py'
if os.path.exists(inf_file):
    with open(inf_file, 'r') as f: code = f.read()
    if "ret['nb_frames'] = int(video_streams[0]" in code:
        new_block = '''    if 'nb_frames' in video_streams[0]:
        ret['nb_frames'] = int(video_streams[0]['nb_frames'])
    else:
        duration = float(ret.get('duration', 0))
        fps_parts = video_streams[0].get('r_frame_rate', '24/1').split('/')
        fps = float(fps_parts[0]) / float(fps_parts[1]) if len(fps_parts) == 2 else 24
        ret['nb_frames'] = int(duration * fps) if duration else 10000'''
        code = code.replace("ret['nb_frames'] = int(video_streams[0]['nb_frames'])", new_block)
        with open(inf_file, 'w') as f: f.write(code)
        print("   ✓ Inference script patched")

print("✅ Ready!")

# CONFIG
MODES = {
    'anime_4k': {'tile': 192, 'tile_pad': 24, 'crf': 22, 'preset': 'fast'},
    'anime_fast': {'tile': 128, 'tile_pad': 16, 'crf': 24, 'preset': 'veryfast'},
    'ultra_fast': {'tile': 64, 'tile_pad': 8, 'crf': 28, 'preset': 'ultrafast'}
}
cfg = MODES[MODE]

# HELPERS
def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

# SCAN
print(f"\n🔍 Scanning {INPUT_DIR}...")
all_files = []
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        for f in files: all_files.append(os.path.join(root, f))

VIDEO_EXT = ('.mp4', '.mkv', '.avi', '.mov', '.webm')
all_vids = sorted([f for f in all_files if f.lower().endswith(VIDEO_EXT)])
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    
    if os.path.exists(out) and os.path.getsize(out) > info['dur']*100000:
        print(f"⏭️ [DONE] {name}")
    elif info['w'] >= 3800:
        print(f"⏭️ [ALREADY 4K] {name}")
    elif info['w'] > 0:
        # Check if segments exist to skip partially
        CACHE_DIR = os.path.join(OUTPUT_DIR, ".cache", name)
        num_segs = math.ceil(info['dur'] / SEGMENT_TIME)
        done_segs = 0
        if os.path.exists(CACHE_DIR):
            for i in range(num_segs):
                if os.path.exists(os.path.join(CACHE_DIR, f"part_{i:03d}.mkv")):
                    done_segs += 1
        
        to_process.append({'path': p, 'name': name, 'info': info, 'out': out})
        status = f"(Resume {done_segs}/{num_segs})" if done_segs > 0 else ""
        print(f"✅ [QUEUE] {name} {status}")

print(f"\n📊 {len(to_process)} videos to process")

# PROCESS LOOP
if to_process:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    TEMP_DIR = '/content/segments'
    os.makedirs(TEMP_DIR, exist_ok=True)
    print(f"\n🚀 BẮT ĐẦU V2.2 (Mode: {MODE})")
    
    for idx, item in enumerate(to_process, 1):
        p, name, info, final_out = item['path'], item['name'], item['info'], item['out']
        print(f"\n{'='*60}")
        print(f"🎬 [{idx}/{len(to_process)}] {name}")
        
        try:
            duration = info['dur']
            num_segs = math.ceil(duration / SEGMENT_TIME)
            if num_segs <= 1: num_segs = 1
            
            print(f"   🧩 Chia thành {num_segs} phần")
            CACHE_DIR = os.path.join(OUTPUT_DIR, ".cache", name)
            os.makedirs(CACHE_DIR, exist_ok=True)
            
            seg_files = []
            all_segs_done = True
            
            for i in range(num_segs):
                seg_name = f"part_{i:03d}.mkv"
                seg_path_drive = os.path.join(CACHE_DIR, seg_name)
                seg_files.append(seg_path_drive)
                
                if os.path.exists(seg_path_drive) and os.path.getsize(seg_path_drive) > 1024*1024:
                    print(f"   ⏭️ Phần {i+1} đã xong")
                else:
                    all_segs_done = False
                    print(f"   ▶️ Đang xử lý phần {i+1}...")
                    
                    # Cleanup temp
                    raw_seg = f"{TEMP_DIR}/raw_{i}.mkv"
                    up_seg = f"{TEMP_DIR}/up_{i}.mkv"
                    if os.path.exists(raw_seg): os.remove(raw_seg)
                    if os.path.exists(up_seg): os.remove(up_seg)
                    
                    start_time = i * SEGMENT_TIME
                    
                    # CUT (Capture output for debug)
                    cmd_cut = f'ffmpeg -y -ss {start_time} -i "{p}" -t {SEGMENT_TIME} -c copy -map 0 "{raw_seg}"'
                    res = subprocess.run(cmd_cut, shell=True, capture_output=True, text=True)
                    
                    if res.returncode != 0:
                        print(f"      ⚠️ Split copy failed (Code {res.returncode}). Error:\n{res.stderr[-300:]}")
                        print(f"      🔄 Retrying with re-encode...")
                        
                        # Fallback: re-encode
                        cmd_cut_re = f'ffmpeg -y -ss {start_time} -i "{p}" -t {SEGMENT_TIME} -c:v libx264 -preset ultrafast -c:a copy "{raw_seg}"'
                        res_re = subprocess.run(cmd_cut_re, shell=True, capture_output=True, text=True)
                        
                        if res_re.returncode != 0:
                            print(f"      ❌ Split re-encode failed:\n{res_re.stderr[-300:]}")
                            raise Exception("Không thể cắt video")

                    # UPSCALE (Capture output để debug)
                    os.chdir(ESRGAN)
                    cmd_up = f'python inference_realesrgan_video.py -n realesr-animevideov3 -i "{raw_seg}" -o "{up_seg}" -s 2 --tile {cfg["tile"]} --tile_pad {cfg["tile_pad"]} --suffix "" --fp32'
                    
                    result = subprocess.run(cmd_up, shell=True, capture_output=True, text=True)
                    if result.returncode != 0:
                        print(f"      ❌ Upscale Failed:")
                        print(result.stderr[-500:])
                        raise Exception("Upscale failed")
                    
                    # ENCODE
                    cmd_enc = f'ffmpeg -y -i "{up_seg}" -c:v libx265 -crf {cfg["crf"]} -preset {cfg["preset"]} -tag:v hvc1 -c:a copy -c:s copy "{seg_path_drive}" -loglevel error'
                    os.system(cmd_enc)
                    
                    # Cleanup
                    if os.path.exists(raw_seg): os.remove(raw_seg)
                    if os.path.exists(up_seg): os.remove(up_seg)
                    print(f"      ✅ Xong phần {i+1}")
            
            # MERGE
            if all_segs_done:
                print("   📦 Đang nối file...")
                list_file = f"{TEMP_DIR}/list.txt"
                with open(list_file, 'w') as f:
                    for sf in seg_files: f.write(f"file '{sf}'\n")
                
                os.system(f'ffmpeg -y -f concat -safe 0 -i "{list_file}" -c copy -map_metadata 0 "{final_out}" -loglevel error')
                print(f"✅ HOÀN THÀNH: {final_out}")
            else:
                print("   ⚠️ Chưa xong, chạy lại để tiếp tục!")

        except Exception as e:
            print(f"   ❌ STOP: {e}")
            
else:
    print("✅ Không có video!")